# [1]환율정보 수집

### (1) 라이브러리 가져오기

In [6]:
# 웹사이트에 데이터 요청
import requests

# 데이터를 표 형태로 관리
import pandas as pd

# 작업 사이의 휴식 시간
import time

# 날짜와 시간을 계산
from datetime import datetime

### (2) 함수 정의

In [7]:
# exrate_get 함수
def exrate_get(ymd_dash, ymd):

    # 환율 데이터가 있는 하나은행 서버 주소
    url="https://www.kebhana.com/cms/rate/wpfxd651_01i_01.do"

    # 서버에 보낼 내용
    payload = dict(ajax="true", tmpInqStrDt=ymd_dash, pbldDvCd="0", inqStrDt=ymd, inqKindCd="1", requestTarget="searchContentDiv")
    
    # url과 상세 조회 조건을 변수 r에 담는다.
    r = requests.get(url, params=payload)

    # 코드 중에서 태그만 골라 데이터 프레임 형태로 변경
    # [0]: 웹페이지 첫 번째 표를 가져오겠다.
    df = pd.read_html(r.text)[0]

    # 가져온 데이터프레임(표)에 날짜 추가
    df.insert(0, '날짜', ymd_dash)
    
    # 완성된 표를 함수 밖으로 내보낸다.
    return df

* OSError 발생
    * 컴퓨터 시스템(운영체제) 수준에서 발생하는 문제
    * 크롤링 시 pd.read_html이 웹사이트 주소나 HTML 텍스트를 읽으려다가 해석할 수 없는 데이터를 만났을 때 발생
* 해결방안으로 StringIO 라이브러리 사용
    * StringIO: 메모리 속에 있는 문자열을 가상의 파일로 변장시켜 준다.

In [22]:
from io import StringIO

In [23]:
# exrate_get 함수
def exrate_get(ymd_dash, ymd):

    # 환율 데이터가 있는 하나은행 서버 주소
    url="https://www.kebhana.com/cms/rate/wpfxd651_01i_01.do"

    # 서버에 보낼 내용
    payload = dict(ajax="true", tmpInqStrDt=ymd_dash, pbldDvCd="0", inqStrDt=ymd, inqKindCd="1", requestTarget="searchContentDiv")
    
    # url과 상세 조회 조건을 변수 r에 담는다.
    r = requests.get(url, params=payload)

    # 코드 중에서 태그만 골라 데이터 프레임 형태로 변경
    # [0]: 웹페이지 첫 번째 표를 가져오겠다.
    df = pd.read_html(StringIO(r.text))[0]    # StringIO 추가

    # 가져온 데이터프레임(표)에 날짜 추가
    df.insert(0, '날짜', ymd_dash)
    
    # 완성된 표를 함수 밖으로 내보낸다.
    return df

# [2]날짜 생성(스케줄러)

* pd.date_range => DatetimeIndex 객체

In [14]:
for date in list(pd.date_range("2026-01-28", "2026-01-23", freq='-1B')):
    print(date, type(date))

2026-01-28 00:00:00 <class 'pandas.Timestamp'>
2026-01-27 00:00:00 <class 'pandas.Timestamp'>
2026-01-26 00:00:00 <class 'pandas.Timestamp'>
2026-01-23 00:00:00 <class 'pandas.Timestamp'>


In [16]:
for date in pd.date_range("2026-01-28", "2026-01-23", freq='-1B'):
    print(date, type(date))

2026-01-28 00:00:00 <class 'pandas.Timestamp'>
2026-01-27 00:00:00 <class 'pandas.Timestamp'>
2026-01-26 00:00:00 <class 'pandas.Timestamp'>
2026-01-23 00:00:00 <class 'pandas.Timestamp'>


In [21]:
result = []
for date in list(pd.date_range("2026-01-28", "2026-01-23", freq='-1B')):
    ymd_dash, ymd = date.strftime("%Y-%m-%d"), date.strftime("%Y%m%d")
    print(ymd_dash, ymd)
    data = exrate_get(ymd_dash, ymd)
    result.append(data)
final_result = pd.concat(result)
final_result

2026-01-28 20260128
2026-01-27 20260127
2026-01-26 20260126
2026-01-23 20260123


날짜            통화       현찰                              송금  \
                          통화     사실 때            파실 때            보낼 때   
                          통화       환율 Spread       환율 Spread     보낼 때   
0   2026-01-28        미국 USD  1458.58   1.75  1408.42   1.75  1447.50   
1   2026-01-28  일본 JPY (100)   950.90   1.75   918.20   1.75   943.70   
2   2026-01-28        유로 EUR  1747.99   1.99  1679.79   1.99  1731.02   
3   2026-01-28        중국 CNY   216.77   5.00   196.13   5.00   208.51   
4   2026-01-28        홍콩 HKD   187.35   1.97   180.13   1.97   185.57   
..         ...           ...      ...    ...      ...    ...      ...   
53  2026-01-23       리비아 LYD     0.00   0.00     0.00   0.00   231.46   
54  2026-01-23      루마니아 RON     0.00   0.00     0.00   0.00   341.48   
55  2026-01-23       미얀마 MMK     0.00   0.00     0.00   0.00     0.69   
56  2026-01-23     에티오피아 ETB     0.00   0.00     0.00   0.00     9.42   
57  2026-01-23    우즈베키스탄 UZS     0.00   0.00     0.00   0.00     0.12   

            외화 수표 파실때   매매 기준율     환가 료율  미화 환산율  
       받을 때 외화 수표 파실때   매매 기준율     환가 료율  미화 환산율  
       받을 때 외화 수표 파실때   매매 기준율     환가 료율  미화 환산율  
0   1419.50   1417.31  1433.50   5.52434  1.0000  
1    925.40    924.75   934.55   2.83136  0.6519  
2   1696.76   1694.90  1713.89   3.91400  1.1956  
3    204.39      0.00   206.45   3.79947  0.1440  
4    181.91    181.69   183.74   4.92233  0.1282  
..      ...       ...      ...       ...     ...  
53   225.98      0.00   228.72   2.97500  0.1572  
54   334.06      0.00   337.77   7.78300  0.2321  
55     0.69      0.00     0.69   1.97500  0.0005  
56     9.20      0.00     9.31   2.97500  0.0064  
57     0.12      0.00     0.12  17.47500  0.0001  

[232 rows x 12 columns]

* 오류 발생

In [17]:
result = []
for date in pd.date_range("2026-01-28", "2026-01-23", freq='-1B'):
    ymd_dash, ymd = date.strftime("%Y-%m-%d"), date.strftime("%Y%m%d")
    print(ymd_dash, ymd)
    data = exrate_get(ymd_dash, ymd)
    result.append(data)
final_result = pd.concat(result)
final_result

2026-01-28 20260128


FileNotFoundError: [Errno 2] No such file or directory: 






<script>
	var regYmdt = "20260128";
	$j("#tmpInqStrDt").val(regYmdt.substring(0,4)+"-"+regYmdt.substring(4,6)+"-"+regYmdt.substring(6,8));
</script>

	 
	<div class="btnRight_tbl mt50">
		<a href="#//HanaBank" class="btnTxt txt" onclick="pbk.foreign.rate.pbld.prs.doExcelDown('Y');" title="현재환율 TXT 다운로드"><span>다운로드</span></a>
		<a href="#//HanaBank" class="btnTxt excel" onclick="pbk.foreign.rate.pbld.prs.doExcelDown('N');" title="현재환율 엑셀 다운로드"><span>다운로드</span></a>
		<a href="#//HanaBank" class="btn_print" onclick="printPopup.open();" title="인쇄하기"><span>인쇄하기</span></a>
	</div>

<div class="printdiv">
	
	<div style="display:none">	
		<h4>현재환율</h4>
	</div>
	
	<p class="txtRateBox">
	<span class="fl">
	<em>기준일</em> : <strong>2026년01월28일</strong>
	<em>고시일시</em> : <strong>
		
			
			
				2026년01월29일
				</strong> <strong>07시28분34초 </strong><strong>(1443회차)</strong></span>
			
		
		
	
	<span c

* (!)final_result의 데이터프레임 내 컬럼 이름에 정리가 필요하다.

* final_result에서 열 제목들 가져오기
    * .columns
    * .index : 행 번호나 이름 가져오기
    * .values : 데이터 값들만 가져오기

In [24]:
final_result.columns

MultiIndex([(       '날짜',          '',          ''),
            (       '통화',        '통화',        '통화'),
            (       '현찰',      '사실 때',        '환율'),
            (       '현찰',      '사실 때',    'Spread'),
            (       '현찰',      '파실 때',        '환율'),
            (       '현찰',      '파실 때',    'Spread'),
            (       '송금',      '보낼 때',      '보낼 때'),
            (       '송금',      '받을 때',      '받을 때'),
            ('외화 수표 파실때', '외화 수표 파실때', '외화 수표 파실때'),
            (   '매매 기준율',    '매매 기준율',    '매매 기준율'),
            (    '환가 료율',     '환가 료율',     '환가 료율'),
            (   '미화 환산율',    '미화 환산율',    '미화 환산율')],
           )

# [3]컬럼 정리

# [4]DB에 저장